In [3]:
import cv2
import numpy as np

In [2]:
def check_overlap_for_2(box1, box2):
    # bbox is TL and BR corner coordinates
    x1, y1, x2, y2 = box1
    a1, b1, a2, b2 = box2

    box1_corners = [
        (x1, y1),  # TL
        (x2, y1),  # TR
        (x1, y2),  # BL
        (x2, y2),  # BR
    ]

    while False:
        corner_overlap_areas = {
            "TL": ((a2 - x1) * (b2 - y1)),
            "TR": ((x1 - a1) * (b2 - y2)),
            "BL": ((a2 - x1) * (y2 - b1)),
            "BR": ((x2 - a1) * (y2 - b1)),
        }

    overlapping_corners = 0
    single_corener = ()  # returns corner for single corner case with small overlap
    for corner in box1_corners:
        x, y = corner
        # check if x1,y1 is in box2
        if (x >= a1 and x <= a2) and (y >= b1 and y <= b2):
            overlapping_corners += 1
            single_corener = (x, y)

    area = 0
    if overlapping_corners > 0:
        # calculate the coordinates of the overlapping rectangle
        x_left = max(x1, a1)
        y_top = max(y1, b1)
        x_right = min(x2, a2)
        y_bottom = min(y2, b2)

        # calculate overlap area
        overlap_width = x_right - x_left
        overlap_height = y_bottom - y_top
        area = overlap_width * overlap_height

    return overlapping_corners, area

In [ ]:

def split_people_bboxes(depth_frame, rgb_frame, bboxes, confs, index):

    # 1 person
    if len(bboxes) == 1:
        print("CASE 1")
        # return depthmap of corresponding image as it is into dataset
        return [depth_frame][rgb_frame]

    # 2 person
    if len(bboxes) == 2:
        print("CASE 2")
        bbox1, bbox2 = bboxes
        overlapping_corners, area = check_overlap_for_2(bbox1, bbox2)

        # no overlap
        if area == 0:
            print("CASE 2.1")
            # sort bboxes to organize by TL corner
            bboxes.sort(key=lambda x: x[0])
            box1, box2 = bboxes
            x1, y1, x2, y2 = box1
            a1, b1, a2, b2 = box2

            # not right, above or below. :. box1 is to the left of box2
            if x2 <= a1:
                print("CASE 2.1.1")
                mid_x = (x2 + a1) // 2

                depth_section1 = depth_frame[:mid_x, :]
                depth_section2 = depth_frame[mid_x:, :]

                rgb_section1 = rgb_frame[:mid_x, :]
                rgb_section2 = rgb_frame[mid_x:, :]

            # box1 is above or below box2
            else:
                print("CASE 2.1.2")
                if y2 >= b1:  # if above
                    print("CASE 2.1.2.1")
                    mid_y = (y2 + b1) // 2

                elif b2 >= y1:  # if below
                    print("CASE 2.1.2.2")
                    mid_y = (y1 + b2) // 2

                depth_section1 = depth_frame[:, :mid_y]
                depth_section2 = depth_frame[:, mid_y:]

                rgb_section1 = rgb_frame[:, :mid_y]
                rgb_section2 = rgb_frame[:, mid_y:]

            return [depth_section1, depth_section2], [rgb_section1, rgb_section2]

        # if overlap
        if area > 0:
            print("CASE 2.2")
            # corner overlap
            if overlapping_corners == 1:
                print("CASE 2.2.1")
                x1, y1, x2, y2 = bboxes[0]
                a1, a2, b1, b2 = bboxes[1]
                # if overlap area is less than 30% area of bbox2
                if area < 0.3 * (a2 - a1) * (b2 - b1) or area < 0.3 * (x2 - x1) * (y2 - y1):
                    print("CASE 2.2.1.1")
                    bboxes.sort(key=lambda x: x[0])
                    box1, box2 = bboxes

                    mid_x = (box1[2] + box2[0]) // 2

                    depth_section1 = depth_frame[:, :mid_x]
                    depth_section2 = depth_frame[:, mid_x:]

                    rgb_section1 = rgb_frame[:, :mid_x]
                    rgb_section2 = rgb_frame[:, mid_x:]

                    return [depth_section1, depth_section2], [
                        rgb_section1,
                        rgb_section2,
                    ]

                # if overlap area is more than 30% area of bbox2
                else:
                    print("CASE 2.2.1.2")
                    box1, box2 = bboxes
                    # box1 is significantly more recognizable than box2
                    crop_bbox1 = depth_frame[y1:y2,x1:x2]
                    crop_bbox2 = depth_frame[b1:b2,a1:a2]
                    
                    # box1 is in front of box2
                    avg_bbox1 = np.average(crop_bbox1)
                    avg_bbox2 = np.average(crop_bbox1)
                    
                    print(avg_bbox1, avg_bbox1)
                    
                    if avg_bbox1 > avg_bbox2 - 20:
                        print("CASE 2.2.1.2.1")
                        # use only box1 
                        depth_section1 = depth_frame[
                            box1[0] : box1[2], box1[1] : box1[3]
                        ]
                        rgb_section1 = rgb_frame[box1[1] : box1[3], box1[0] : box1[2]]

                        return [depth_section1],[rgb_section1]
                    
                    # box2 is in front of box1
                    elif avg_bbox2 > avg_bbox1 - 20:
                        print("CASE 2.2.1.2.2")
                        # use only box1
                        depth_section1 = depth_frame[
                            box2[1] : box2[3], box2[0] : box2[2]
                        ]
                        rgb_section1 = rgb_frame[box2[1] : box2[3], box2[0] : box2[2]]

                        return [depth_section1],[rgb_section1]

                    else:
                        print("CASE 2.2.1.2.3")
                        # both box are similarly recognizable and have significant overlap
                        while 0:
                            depth_section1 = depth_frame[
                                box1[1] : box1[3], box1[0] : box1[2]
                            ]
                            depth_section2 = depth_frame[
                                box2[1] : box2[3], box2[0] : box2[2]
                            ]

                            rgb_section1 = rgb_frame[box1[1] : box1[3], box1[0] : box1[2]]
                            rgb_section2 = rgb_frame[box2[1] : box2[3], box2[0] : box2[2]]

                            return [depth_section1, depth_section2], [
                                rgb_section1,
                                rgb_section2
                            ]
                        return [],[]
            # edge overlap
            elif overlapping_corners == 2:
                print("CASE 2.2.2")
                
                if area < 0.3 * (a2 - a1) * (b2 - b1) or area < 0.3 * (x2 - x1) * (y2 - y1):
                    pass
                

            # full image overlap
            else:
                print("CASE 2.2.3")
                raise Exception
                pass

    # 3 person
    if len(bboxes) > 2:
        print("CASE 3")
        pass

    if bboxes == []:
        print("CASE 4")
        # skip image
        pass
    pass